In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt 
import seaborn as sns 

Customer Churn Prediction Using Artificial Neural Network (ANN)

Customer churn prediction is to measure why customers are leaving a business. In this tutorial we will be looking at customer churn in telecom business. We will build a deep learning model to predict the churn and use precision,recall, f1-score to measure performance of our model

In [5]:
dataset = pd.read_csv(r"/content/drive/MyDrive/churn.csv")
dataset.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,CallService,MultipleConnections,InternetConnection,OnlineSecurity,OnlineBackup,DeviceProtectionService,TechnicalHelp,OnlineTV,OnlineMovies,Agreement,BillingMethod,PaymentMethod,MonthlyServiceCharges,TotalAmount,Churn
0,2907-ILJBN,Female,0.0,Yes,Yes,11.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.6,233.90,No
1,3896-RCYYE,Female,0.0,No,No,67.0,No,No phone service,DSL,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),53.4,3579.15,No


In [6]:
dataset = dataset.drop(["customerID"],axis=1)

In [7]:
dataset.head(1)

,gender,SeniorCitizen,Partner,Dependents,tenure,CallService,MultipleConnections,InternetConnection,OnlineSecurity,OnlineBackup,DeviceProtectionService,TechnicalHelp,OnlineTV,OnlineMovies,Agreement,BillingMethod,PaymentMethod,MonthlyServiceCharges,TotalAmount,Churn
0,Female,0.0,Yes,Yes,11.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.6,233.9,No


In [8]:
dataset.isnull().sum()

gender                     0
SeniorCitizen              0
Partner                    0
Dependents                 0
tenure                     0
CallService                0
MultipleConnections        0
InternetConnection         0
OnlineSecurity             0
OnlineBackup               0
DeviceProtectionService    0
TechnicalHelp              0
OnlineTV                   0
OnlineMovies               0
Agreement                  0
BillingMethod              0
PaymentMethod              0
MonthlyServiceCharges      0
TotalAmount                0
Churn                      0
dtype: int64

In [9]:
dataset.shape

(12335, 20)

In [10]:
dataset.MonthlyServiceCharges = dataset.MonthlyServiceCharges.astype("object")
dataset.SeniorCitizen = dataset.SeniorCitizen.astype("int")
dataset.TotalAmount = dataset.TotalAmount.astype("object")
dataset.tenure = dataset.tenure.astype("int")

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12335 entries, 0 to 12334
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gender                   12335 non-null  object
 1   SeniorCitizen            12335 non-null  int64 
 2   Partner                  12335 non-null  object
 3   Dependents               12335 non-null  object
 4   tenure                   12335 non-null  int64 
 5   CallService              12335 non-null  object
 6   MultipleConnections      12335 non-null  object
 7   InternetConnection       12335 non-null  object
 8   OnlineSecurity           12335 non-null  object
 9   OnlineBackup             12335 non-null  object
 10  DeviceProtectionService  12335 non-null  object
 11  TechnicalHelp            12335 non-null  object
 12  OnlineTV                 12335 non-null  object
 13  OnlineMovies             12335 non-null  object
 14  Agreement                12335 non-nul

In [34]:
#frequency of categories
for col in dataset:
    print('\n%s column: '%col)
    print(dataset[col].value_counts())


gender column: 
0    6216
1    6119
Name: gender, dtype: int64

SeniorCitizen column: 
0    10769
1     1566
Name: SeniorCitizen, dtype: int64

Partner column: 
1    6230
0    6105
Name: Partner, dtype: int64

Dependents column: 
0    7974
1    4361
Name: Dependents, dtype: int64

tenure column: 
0.000000    1084
1.000000     485
0.014085     434
0.028169     374
0.042254     317
            ... 
0.605634      92
0.901408      91
0.816901      90
0.830986      86
0.760563      84
Name: tenure, Length: 72, dtype: int64

CallService column: 
1    10933
0     1402
Name: CallService, dtype: int64

MultipleConnections column: 
0    7377
1    4958
Name: MultipleConnections, dtype: int64

InternetConnection column: 
1    4995
0    4130
2    3210
Name: InternetConnection, dtype: int64

OnlineSecurity column: 
0    9006
1    3329
Name: OnlineSecurity, dtype: int64

OnlineBackup column: 
0    8345
1    3990
Name: OnlineBackup, dtype: int64

DeviceProtectionService column: 
0    8414
1    3921
N

In [13]:
dataset.replace('No phone service','No',inplace=True)
dataset.replace('No internet service','No',inplace=True)

## Label Encoding

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [15]:
dataset[dataset.select_dtypes(include=['object']).columns] = dataset[dataset.select_dtypes(include=['object']).columns].apply(le.fit_transform)

In [16]:
dataset.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,CallService,MultipleConnections,InternetConnection,OnlineSecurity,OnlineBackup,DeviceProtectionService,TechnicalHelp,OnlineTV,OnlineMovies,Agreement,BillingMethod,PaymentMethod,MonthlyServiceCharges,TotalAmount,Churn
0,0,0,1,1,11,1,0,2,0,0,0,0,0,0,1,0,3,20.6,233.90,0
1,0,0,0,0,67,0,0,0,0,0,1,1,1,1,0,1,1,53.4,3579.15,0


In [17]:
X=dataset.drop(['Churn'],axis=1)
X

,gender,SeniorCitizen,Partner,Dependents,tenure,CallService,MultipleConnections,InternetConnection,OnlineSecurity,OnlineBackup,DeviceProtectionService,TechnicalHelp,OnlineTV,OnlineMovies,Agreement,BillingMethod,PaymentMethod,MonthlyServiceCharges,TotalAmount
0,0,0,1,1,11,1,0,2,0,0,0,0,0,0,1,0,3,20.600000,233.900000
1,0,0,0,0,67,0,0,0,0,0,1,1,1,1,0,1,1,53.400000,3579.150000
2,0,0,1,0,59,1,0,2,0,0,0,0,0,0,2,0,0,18.400000,1057.850000
3,1,0,1,1,67,1,1,2,0,0,0,0,0,0,2,0,3,26.300000,1688.900000
4,0,0,0,0,11,1,1,1,0,0,0,0,0,0,0,0,2,75.250000,888.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12330,1,0,0,1,2,1,0,1,0,0,0,0,0,0,0,1,3,40.151251,76.135546
12331,0,0,0,0,63,0,0,1,1,0,1,0,0,0,0,0,1,96.252187,6310.292602
12332,1,0,0,1,49,0,0,1,1,1,1,0,0,0,0,0,1,70.445975,4091.168541
12333,1,0,1,1,20,1,0,2,0,0,0,0,0,0,0,1,3,55.819099,560.153643


In [18]:
y=dataset.loc[:,'Churn']
y

0        0
1        0
2        0
3        0
4        0
        ..
12330    1
12331    1
12332    1
12333    1
12334    1
Name: Churn, Length: 12335, dtype: int64

In [19]:
dataset.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,CallService,MultipleConnections,InternetConnection,OnlineSecurity,OnlineBackup,DeviceProtectionService,TechnicalHelp,OnlineTV,OnlineMovies,Agreement,BillingMethod,PaymentMethod,MonthlyServiceCharges,TotalAmount,Churn
0,0,0,1,1,11,1,0,2,0,0,0,0,0,0,1,0,3,20.60,233.90,0
1,0,0,0,0,67,0,0,0,0,0,1,1,1,1,0,1,1,53.40,3579.15,0
2,0,0,1,0,59,1,0,2,0,0,0,0,0,0,2,0,0,18.40,1057.85,0
3,1,0,1,1,67,1,1,2,0,0,0,0,0,0,2,0,3,26.30,1688.90,0
4,0,0,0,0,11,1,1,1,0,0,0,0,0,0,0,0,2,75.25,888.65,0
5,0,0,1,1,36,1,1,0,1,1,1,1,1,1,2,1,1,92.90,3379.25,0
6,0,0,1,1,49,1,0,0,1,0,1,1,0,0,0,0,0,61.75,3024.15,0
7,0,0,0,0,54,1,0,1,1,1,0,0,1,1,1,1,1,100.05,5299.65,0
8,1,0,0,0,26,1,1,1,0,0,1,1,0,0,0,1,3,84.30,2281.60,0
9,1,0,0,0,19,1,0,2,0,0,0,0,0,0,1,0,3,18.80,279.20,0


In [20]:
cols_to_scale = ['tenure','MonthlyServiceCharges','TotalAmount']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataset[cols_to_scale] = scaler.fit_transform(dataset[cols_to_scale])

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=10)

### Apply Logistics regession

In [22]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()

In [23]:
model = lg.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [24]:
pred = model.predict(X_test)

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [26]:
tab = confusion_matrix(pred,y_test)
print("Confusion Matrix:\n",tab)
# evaluate predictions
acc = (accuracy_score(y_test, pred))*100
print("\nAccuracy: %.2f%%\n" % acc)
print(classification_report(y_test,pred))

Confusion Matrix:
 [[916 324]
 [445 782]]

Accuracy: 68.83%

              precision    recall  f1-score   support

           0       0.74      0.67      0.70      1361
           1       0.64      0.71      0.67      1106

    accuracy                           0.69      2467
   macro avg       0.69      0.69      0.69      2467
weighted avg       0.69      0.69      0.69      2467



In [27]:
X_train.shape

(9868, 19)

## Apply ANN

In [28]:
import tensorflow as tf
from tensorflow import keras

In [32]:
model1 = keras.Sequential([
    keras.layers.Dense(18, input_shape=(19,), activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

#opt = keras.optimizers.Adam(learning_rate=0.01)

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [33]:
model1.fit(X_train,y_train, epochs=100)

Epoch 1/100
309/309 [==============================] - 1s 2ms/step - loss: 7.0789 - accuracy: 0.5723
Epoch 2/100
309/309 [==============================] - 1s 2ms/step - loss: 0.8273 - accuracy: 0.6552
Epoch 3/100
309/309 [==============================] - 1s 2ms/step - loss: 0.9289 - accuracy: 0.6568
Epoch 4/100
309/309 [==============================] - 1s 2ms/step - loss: 0.8850 - accuracy: 0.6543
Epoch 5/100
309/309 [==============================] - 1s 2ms/step - loss: 1.2698 - accuracy: 0.6488
Epoch 6/100
309/309 [==============================] - 1s 2ms/step - loss: 0.6936 - accuracy: 0.6849
Epoch 7/100
309/309 [==============================] - 1s 2ms/step - loss: 0.9525 - accuracy: 0.6509
Epoch 8/100
309/309 [==============================] - 1s 2ms/step - loss: 0.8332 - accuracy: 0.6725
Epoch 9/100
309/309 [==============================] - 1s 2ms/step - loss: 0.7553 - accuracy: 0.6658
Epoch 10/100
309/309 [==============================] - 1s 2ms/step - loss: 0.9233 - accura